# Generate Fringes from datasets

Edited by Louise 03/2021

In this notebook we produce the fringes files from the raw QUBIC datasets from 2020/01/13, 27/10/2020 and 28/10/2020. 

We mainly use the libraries `fringes_lib.py` and `selfcal_lib.py` from QUBIC soft.

Fringes are saved into **fits** files. PDF with plots can also be produced.
 

In [ ]:
%matplotlib inline
%matplotlib notebook

from importlib import reload

# Specific science modules
import scipy
import glob
import numpy as np
import matplotlib.pyplot as plt

import qubic
from qubicpack.utilities import Qubic_DataDir
from qubicpack import qubicpack as qp
from qubicpack.qubicfp import qubicfp
import qubic.fibtools as ft
from qubic import fringes_lib as flib
from qubic import selfcal_lib as scal

rc('figure',figsize=(12,6))
rc('font',size=12)

## Make a QUBIC instrument and get TES coordinates

In [ ]:
basedir = Qubic_DataDir(datafile='instrument.py', )
dictfilename = basedir + '/dicts/global_source_oneDet.dict'
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)
q = qubic.QubicInstrument(d)

In [ ]:
# Try just for one TES
TES = 46
ASIC = 1 # If q is a TD, it should be 1 or 2
xONAFP, yONAFP, FP_index, index_q = scal.TES_Instru2coord(TES, ASIC, q, frame='ONAFP')

# Get coordinates for all TES
xTES, yTES, allFP_index, allindex_q = scal.get_TES_Instru_coords(q, frame='ONAFP')

## Look at the dataset

In [ ]:
global_dir = '/home/lmousset/QUBIC/Qubic_work/Calibration/datas/Fringes/'
# date = '2020-10-27'
# date = '2020-01-13'
date = '2019-06-07'

print(global_dir)
vtes = 5
eco = 1

out_dir = global_dir 
# Check that out_dir exists, if not try to create
try:
    os.makedirs(out_dir, exist_ok = True)
except OSError as e:
    print('Unable to create Output Directory: ' + out_dir)

In [ ]:
# Get the datasets
def get_fringes_datasets(rep, keyword, q):
    """
    Get the dataset with fringes measurements and classify the baselines
    by type of equivalency.
    Parameters
    ----------
    rep: str
        Repository with the data.
    keyword: str
        Key word in the data folders you want to look at.
    q: QubicInstrument

    Returns
    -------
    datasets: List with all the data folders
    allBLs: List with all the baselines.
    allNcycles: List with the Ncycles corresponding to each data folder.
    allstable_time: List with the waiting times corresponding to each data folder.
    """
    if rep[-1] != '/':
        rep += '/'

    datasets = np.sort(glob.glob(rep + '*' + keyword + '*'))
    print('Number of folders in the dataset:', len(datasets))

    allBLs, allNcycles, allstable_time = [], [], []
    for ds in datasets:
        strs = ds.split('_')
        print(strs)
        allNcycles.append(20)
        allstable_time.append(1000.)
#         allNcycles.append(float(strs[strs.index('ncycles') + 1]))
#         allstable_time.append(float(strs[strs.index('wt') + 1]))
        allBLs.append([int(strs[-2]), int(strs[-1])])

    return datasets, allBLs, allNcycles, allstable_time

datasets, allBLs, allNcycles, allstable_time = get_fringes_datasets(global_dir + date, '*' + 'RF_switch', q)
BLs_sort, BLs_type = scal.find_equivalent_baselines(allBLs, q)

nBLs = len(allBLs)
print('nBLs:', nBLs)
print('Ncycles:', allNcycles)
print('Stable time:', allstable_time)
print('Baselines:', allBLs)
print('BL sorted:', BLs_sort)
print('BL Type:', BLs_type)   

# Plot the baselines
scal.plot_BLs_eq(allBLs, BLs_sort, q)

## Simulation to compare 

In [ ]:
rep = '/home/lmousset/QUBIC/Qubic_work/Calibration/RF_switch/simu_creidhe_aberrations/creidhe_files/'
for baseline in allBLs[5:6]:
    Model_May = scal.Model_Fringes_Maynooth(q, baseline,
                                            rep=rep,
                                            theta_source=np.deg2rad(0.),
                                            nu_source=150e9,
                                            frame='ONAFP')

    xONAFP, yONAFP, fringes = Model_May.get_fringes_from_combination(verbose=True)

#     print('fringes shape:', fringes.shape)




In [ ]:
lim = np.max(np.abs(fringes))
fig = plt.figure(figsize=(7, 7))
ax = fig.gca()
scal.scatter_plot_FP(q, xONAFP, yONAFP, fringes, frame='ONAFP', s=350, fig=fig, ax=ax, unit=None,
                    title=f'Simulation - Baseline {allBLs[5]}', cmap='bwr',
                    vmin=-lim, vmax=lim, fontsize=20)
fig.tight_layout()
# fig.savefig('/home/lmousset/QUBIC/Images/simu_fringes_49-51_Maynooth.pdf')

In [ ]:
for baseline in allBLs[5:6]:
    Model_QS = scal.Model_Fringes_QubicSoft(q, baseline,
                                            theta_source=0., phi_source=0.,
                                            nu_source=150e9, spec_irrad_source=1.,
                                            frame='ONAFP', external_A=None, hwp_position=0)

    xONAFP, yONAFP, fringes = Model_QS.get_fringes_from_combination(doplot=True, verbose=True, norm=None, unit=None, s=180)

#     print('fringes shape:', fringes.shape)


lim = np.max(np.abs(fringes))
fig = plt.figure(figsize=(7, 7))
ax = fig.gca()
scal.scatter_plot_FP(q, xONAFP, yONAFP, fringes, frame='ONAFP', s=350, fig=fig, ax=ax, unit=None,
                    title=f'Simulation - Baseline {allBLs[5]}', cmap='bwr',
                    vmin=-lim, vmax=lim)


In [ ]:
baseline=allBLs[5]
Model_ana = scal.Model_Fringes_Ana(q, baseline,
                                    theta_source=0.,
                                    nu_source=150e9,
                                    frame='ONAFP')

xONAFP, yONAFP, fringes = Model_ana.get_fringes(times_gaussian=True)

fig = plt.figure(figsize=(7, 7))
ax = fig.gca()
scal.scatter_plot_FP(q, xONAFP, yONAFP, fringes, frame='ONAFP', s=350, fig=fig, ax=ax, unit=None,
                    title=f'Simulation - Baseline {allBLs[5]}', cmap='bwr')


## Fringes Analysis

#### Analysis

In [ ]:
allBLs

In [ ]:
allm_points, allerr_m_points = [], []
allMcycles, allerr_Mcycles = [], []
allfringes1D, allerr_fringes1D = [], []
allfringes1D_percycle, allerr_fringes1D_percycle = [], []
allTODresiduals = []
allmask_bad_TES = []

# Loop on the BLs
for i, BL in enumerate(allBLs[-1]):
    i=7
    print(f'\n ===================== Measurement {i+1}/{nBLs} - BL {BL} starting ===================')   
    print("Data folder:", datasets[i])

    # Analysis
    Fringes_Ana = flib.FringesAnalysis(datasets[i], date, q, BL, stable_time=1., ncycles=20, 
                                       refTESnum=None, refASICnum=None, fraction_bad_TES=0.75, 
                                       sigma_conv_astropy=0.5, verbose=True, sort_TES=True)
#     allmask_bad_TES.append(Fringes_Ana.mask_bad_TES)
    Fringes_Ana.plot_TOD()
#     m_points, err_m_points, Mcycles, err_Mcycles, \
#     fringes1D, err_fringes1D, fringes1D_percycle, err_fringes1D_percycle = Fringes_Ana.analyse_fringes(
#                               doplotTESsort=[0, 30],
#                               median=True,
#                               Ncycles_to_use=None,
#                               remove_median_allh=True,
#                               rm_slope_percycle=False)

#     allm_points.append(m_points)
#     allerr_m_points.append(err_m_points)
#     allMcycles.append(Mcycles)
#     allerr_Mcycles.append(err_Mcycles)
    
#     allfringes1D.append(fringes1D)
#     allerr_fringes1D.append(err_fringes1D)
    
#     allfringes1D_percycle.append(fringes1D_percycle)
#     allerr_fringes1D_percycle.append(err_fringes1D_percycle)

## Make some plots

#### Plot the fringes on the FP

In [ ]:
k = 0
cmap_bwr = flib.make_cmap_nan_black('bwr')

fig, axs = plt.subplots(1, 2)
ax1, ax2 = axs.ravel()
# Scatter plot
flib.plot_fringes_scatter(q, xTES, yTES, allfringes1D[k]* allmask_bad_TES[k], s=150, 
                          fig=fig, ax=ax1, cmap=cmap_bwr)

# Imshow after a convolution with Astropy (gaussian kernel)
fringes2D = flib.make2Dfringes_data(allfringes1D[k] * allmask_bad_TES[k])
fringes2D_conv = flib.astropy_convolution(fringes2D, sigma=0.7)
flib.plot_fringes_imshow(fringes2D_conv, normalize=True, fig=fig, ax=ax2, cmap=cmap_bwr, 
                         title='Gaussian convolution', mask=flib.make_mask2D_thermometers_TD())


#### Plot the fringes value as a function of the cycle

In [ ]:
k = 0
ASIC = 1
for TES in [70, 35, 95, 100]:
    Fringes_Ana._plot_fringes_measurement_perTES(allfringes1D[k], 
                                                 allerr_fringes1D[k], 
                                                 allfringes1D_percycle[k], 
                                                 allerr_fringes1D_percycle[k],
                                                 TES=TES,
                                                 ASIC=ASIC)


#### Plot time response with and without bad TES 

In [ ]:
ctimes = Fringes_Ana.ctimes
plt.figure()
plt.hist(np.ravel(ctimes), bins=15, alpha=0.4, label='with all TES')
plt.hist(np.ravel(ctimes)*Fringes_Ana.mask_bad_TES, bins=15, alpha=0.4, label='without the 25% worst TES')
plt.legend()
plt.xlabel('Time response [s]', fontsize=16)

#### Look at some TODs

In [ ]:
for TES in range(1):
    Fringes_Ana.plot_TOD(ASIC=1, TES=70)
    plt.xlabel('Time [s]', fontsize=16)
    plt.ylabel('TOD [A.U.]', fontsize=16)

#### Look at the average made on each cycle for some TES

In [ ]:
for TES in range(2):
    Fringes_Ana.plot_average_over_steps(m_points[TES], err_m_points[TES], Mcycles[TES], err_Mcycles[TES])

## Save the data

In [ ]:
Save_class = flib.SaveFringesFitsPdf(q, date, allBLs[:], allstable_time[:], allNcycles[:], xTES, yTES, 
                                       allfringes1D, allerr_fringes1D, allmask_bad_TES=allmask_bad_TES,
                                       nsteps=6, ecosorb='yes', frame='ONAFP')

save_name = 'Fringes_' + date + f'_{Save_class.nBLs}BLs_RemoveSlopePerTES_medianTrue_refTESautomatic_maskbadTES0.75'
Save_class.save_fringes_pdf_plots(global_dir, save_name=save_name + '.pdf', mask=flib.make_mask2D_thermometers_TD())

Save_class.write_fits_fringes(global_dir, save_name=save_name + '.fits')


## Find the bad TES using all images

In [ ]:
# Load a fits file already saved with several images
myfringes = 'Fringes_2020-10-27_12BLs_RemoveSlopePerTES_medianTrue_refTESautomatic_maskbadTES0.75.fits'

header, fdict = flib.read_fits_fringes(global_dir + myfringes)
print(fdict.keys())
allmask_bad_TES = fdict['MASK_BAD_TES']


In [ ]:
cmap = flib.make_cmap_nan_black('bwr')
flib.plot_fringes_scatter(q, xTES, yTES, allmask_bad_TES[0], normalize=False, cbar=False, s=180, cmap=cmap)

In [ ]:
thecond = np.arange(2, 13)
nbad = []

for cond in thecond:
    the_mask = flib.decide_bad_TES(allmask_bad_TES, condition=cond)
#     print(the_mask)
    nbad.append(int(256 - np.nansum(the_mask)))

plt.figure()
plt.plot(thecond, nbad, 'bo')
plt.xlabel('Number of images where the TES is NAN')
plt.ylabel('Number of bad TES')
plt.grid()


In [ ]:
the_mask = flib.decide_bad_TES(allmask_bad_TES, condition=9)
nbad = int(256 - np.nansum(the_mask))

print(the_mask)

badTES = flib.give_index_bad_TES(the_mask)
print(badTES.T)

flib.plot_fringes_scatter(q, xTES, yTES, the_mask, normalize=False, s=140, cbar=False)

In [ ]:
# Plot TOD of the one bad TES on all images
ibad = 0 # Bad TES index
for i, BL in enumerate(allBLs[:]):
    print(f'\n ===================== Measurement {i+1}/{nBLs} - BL {BL} starting ===================')   
    print("Data folder:", datasets[i])

    Fringes_Ana = flib.FringesAnalysis(datasets[i], date, q, BL, stable_time=5., ncycles=20, 
                                       refTESnum=95, refASICnum=1, fraction_bad_TES=0.75, 
                                       sigma_conv_astropy=0.7, sort_TES=False, verbose=True)

    Fringes_Ana.plot_TOD(ASIC=badTES[ibad, 1], TES=badTES[ibad, 0])

In [ ]:
# Plot TOD of all bad TES on one image
img = 6 # image index
 
print("Data folder:", datasets[img])

Fringes_Ana = flib.FringesAnalysis(datasets[img], date, q, BL, stable_time=5., ncycles=20, 
                                   refTESnum=95, refASICnum=1, fraction_bad_TES=0.75, 
                                   sigma_conv_astropy=0.7, sort_TES=False, verbose=True)
for ibad in range(nbad):
    Fringes_Ana.plot_TOD(ASIC=badTES[ibad, 1], TES=badTES[ibad, 0])